## Comparing the use of "HKG" or "CHN" source for mixing matrices for HCMC's model

In [ ]:
import numpy as np

from autumn.tools.inputs.social_mixing.queries import get_prem_mixing_matrices
from autumn.tools.inputs.social_mixing.build_synthetic_matrices import build_synthetic_matrices
from autumn.tools.inputs.social_mixing.constants import LOCATIONS

import matplotlib.pyplot as plt
import matplotlib.cm as cm

In [ ]:
model_iso3 = "VNM" # model for Vietnam
source_iso3_curr = "HKG" # current source of mixing matrices (Hong Kong)
source_iso3_prev = "CHN" # previous source of mixing matrices (China)
age_adjust = True

AGEGROUP_STRATA = [str(5 * i) for i in range(16)]


prem_matrices_curr = prem_matrices_prev = get_prem_mixing_matrices(model_iso3)
new_matrices_curr = build_synthetic_matrices(model_iso3, source_iso3_curr, 
                                             AGEGROUP_STRATA, age_adjust)
new_matrices_prev = build_synthetic_matrices(model_iso3, source_iso3_prev,
                                             AGEGROUP_STRATA, age_adjust)


# normalise new_matrices
lambdas_prem_curr, _ = np.linalg.eig(prem_matrices_curr["all_locations"])
sp_radius_prem_curr = max([abs(l) for l in lambdas_prem_curr])

lambdas_prem_prev, _ = np.linalg.eig(prem_matrices_prev["all_locations"])
sp_radius_prem_prev = max([abs(l) for l in lambdas_prem_prev])

lambdas_new_curr, _ = np.linalg.eig(new_matrices_curr["all_locations"])
sp_radius_new_curr = max([abs(l) for l in lambdas_new_curr])

lambdas_new_prev, _ = np.linalg.eig(new_matrices_prev["all_locations"])
sp_radius_new_prev = max([abs(l) for l in lambdas_new_prev])

for location in LOCATIONS:
    prem_matrices_curr[location] = prem_matrices_curr[location] * sp_radius_new_curr / sp_radius_prem_curr
    prem_matrices_prev[location] = prem_matrices_prev[location] * sp_radius_new_prev / sp_radius_prem_prev
    
matrices = {
    "prem_curr": prem_matrices_curr,
    "prem_prev": prem_matrices_prev,
    "new_curr": new_matrices_curr,
    "new_prev": new_matrices_prev
}


In [ ]:
def compare_by_location(location, add_values=False):
    fig = plt.figure()
    fig.set_size_inches(24, 16)
    
    ticks = [2*i - .5 for i in range(8)]
    labels = [str(int(2*i*5)) for i in range(8)] 
    max_value = max(matrices["prem_curr"][location].max(), matrices["prem_prev"][location].max(),
                    matrices["new_curr"][location].max(), matrices["new_prev"][location].max())
    
    i = 0
    for key, mats in matrices.items():
        i += 1
        ax = fig.add_subplot(1, 4, i)
        
        #ax1 = fig.add_subplot(121)
        ax.imshow(np.transpose(mats[location]), cmap=cm.hot, vmin = 0, vmax=max_value, origin='lower')
        
        if add_values:
            n = mats[location].shape[0]
            for _i in range(n):
                for _j in range(n):
                    plt.text(_i - .5, _j, round(mats[location][_i, _j],1), color="green", fontsize=7)

        ax.set_title(key)
        plt.xlabel("index")
        plt.ylabel("n_contacts")
        plt.xticks(ticks, labels)
        plt.yticks(ticks, labels)


    plt.show()

    
    
def compare_location_contributions():
    i_ages_illustrate = [2, 6, 10, 14]
    
    colors=["coral","cornflowerblue","palegreen","bisque"]
    
    locations_notall = [l for l in LOCATIONS if l != "all_locations"]
    
    for i_age in i_ages_illustrate:
        print(f"Age group starting {i_age * 5} years old")
        fig = plt.figure()
        fig.set_size_inches(24, 16)
        i = 0
        for key, mat in matrices.items():
            i += 1
            ax = fig.add_subplot(1, 4, i) 
            n_contacts = []
            for loc in locations_notall:
                n_contacts.append(sum(mat[loc][i_age,]))            
                
            ax.pie(n_contacts, labels=locations_notall, autopct='%1.1f%%', shadow=True, startangle=90, colors=colors)
            ax.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.

        plt.show()
    

In [ ]:
# Create matrix outputs to look as similar as possible to previous Vic figure
def mixing_matrix_by_location(matrix, fig_title: str): 
    fig = plt.figure(dpi=200)
    fig.suptitle(fig_title, fontsize=13)
    reordered_locations = ("all_locations", "home", "work", "other_locations", "school")
    positions = [1, 2, 3, 5, 6]
    for i_loc, location in zip(positions, reordered_locations):
        ax = fig.add_subplot(2, 3, i_loc)
        ax.imshow(
            np.flipud(np.transpose(matrix[location])), 
            cmap=cm.hot, 
            vmin=0,
            vmax=matrix[location].max(), 
            origin='lower'
        )
        ax.set_title(location.replace("_", " "), fontsize=11)
        ax.set_xticks([])
        ax.set_yticks([])

In [ ]:
mixing_matrix_by_location(new_matrices_curr, "Synthetic mixing matrix for HCMC from using HKG source")

In [ ]:
mixing_matrix_by_location(new_matrices_prev, "Synthetic mixing matrix for HCMC from using CHN source")

In [ ]:
for location in LOCATIONS:
    print(location)
    compare_by_location(location, False)

In [ ]:
for location in LOCATIONS:
    print(location)
    compare_by_location(location, True)

In [ ]:
compare_location_contributions()

In [ ]:
def plot_total_contacts():
    fig = plt.figure(figsize=(12, 5))
    ax = fig.add_axes([0,0,1,1])
    barWidth = .15
    
    prem_curr_s, prem_prev_s, new_curr_s, new_prev_s = [], [], [], []
    for i_age_group in range(16):
        prem_curr_s.append(sum(prem_matrices_curr["all_locations"][i_age_group,]))
        prem_prev_s.append(sum(prem_matrices_prev["all_locations"][i_age_group,]))
        new_curr_s.append(sum(new_matrices_curr["all_locations"][i_age_group,]))
        new_prev_s.append(sum(new_matrices_prev["all_locations"][i_age_group,]))
        
    # Set position of bar on X axis
    br1 = np.arange(len(prem_curr_s))
    br2 = [x + barWidth for x in br1]
    br3 = [x + barWidth for x in br2]
    br4 = [x + barWidth for x in br3]

    plt.bar(br1, prem_curr_s, color ='coral', width = barWidth,
        edgecolor ='grey', label ='prem_curr')
    plt.bar(br2, new_curr_s, color ='cornflowerblue', width = barWidth,
        edgecolor ='grey', label ='new_curr')
    plt.bar(br3, prem_prev_s, color ='palegreen', width = barWidth,
        edgecolor ='grey', label ='prem_prev')
    plt.bar(br4, new_prev_s, color ='bisque', width = barWidth,
            edgecolor ='grey', label ='new_prev')
    
    # Adding Xticks
    plt.xlabel('Index age', fontsize = 12)
    plt.ylabel('Total contacts', fontsize = 12)
    plt.xticks([r + barWidth for r in range(len(prem_curr_s))],
            [str(int(5*i)) for i in range(16)])
    
    plt.legend()
    plt.show()

plot_total_contacts()



In [ ]:
def print_brief_mixing_matrices(matrices, matrices_title: str):
    print(matrices_title)
    matrices_im_young = matrices["other_locations"][2,15]
    matrices_im_old = matrices["other_locations"][15, 2]
    print(f"I am young, N contacts with old people: {matrices_im_young}")
    print(f"I am old, N contacts with young people: {matrices_im_old}")

In [ ]:
print_brief_mixing_matrices(prem_matrices_curr, "Current Prem (HKG)")
print_brief_mixing_matrices(prem_matrices_prev, "Previous Prem (CHN)")

In [ ]:
print_brief_mixing_matrices(new_matrices_curr, "Current New (HKG)")
print_brief_mixing_matrices(new_matrices_prev, "Previous New (CHN)")

In [ ]:
# Save as csv
for loc in LOCATIONS:
    m = new_matrices_curr[loc]
    np.savetxt(f"{loc}.csv", m, delimiter=",")